In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

%matplotlib inline

# Loading, Describing and Cleaning

In [152]:
loans = pd.read_csv('data/kiva_loans.csv')
loans.head()

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01


In [153]:
loans.describe()

,id,funded_amount,loan_amount,partner_id,term_in_months,lender_count
count,6.712050e+05,671205.000000,671205.000000,657698.000000,671205.000000,671205.000000
mean,9.932486e+05,785.995061,842.397107,178.199616,13.739022,20.590922
std,1.966113e+05,1130.398941,1198.660073,94.247581,8.598919,28.459551
min,6.530470e+05,0.000000,25.000000,9.000000,1.000000,0.000000
25%,8.230720e+05,250.000000,275.000000,126.000000,8.000000,7.000000
50%,9.927800e+05,450.000000,500.000000,145.000000,13.000000,13.000000
75%,1.163653e+06,900.000000,1000.000000,204.000000,14.000000,24.000000
max,1.340339e+06,100000.000000,100000.000000,536.000000,158.000000,2986.000000


- The mean of funded_amount which is The amount disbursed by Kiva to the field agent(USD) is lower than loan_amount which is The amount disbursed by the field agent to the borrower(USD), and from the min in describe function above, it seems that there's a case(s) where Kiva didn't disburse money to the field agent but the field agent always disbursed money.
- The data is right skewed since the mean is greater than the median (50% percentile), which means than on average, more loan amounts disbursed are greater than 450 USD.
- On average, it takes approximately 13 months to disburse a loan with some as fast as 1 month and other taking longer like 158 months. I think this value represent the total amount of time it took to disburse the full loan amount.
- The average number of lenders who contributed to a loan is approximately 20, with some as low as 0 and some as high as 2986.
- Some partners are lacking id.

In [154]:
# check for nulls
loans.isnull().any()

id                    False
funded_amount         False
loan_amount           False
activity              False
sector                False
use                    True
country_code           True
country               False
region                 True
currency              False
partner_id             True
posted_time           False
disbursed_time         True
funded_time            True
term_in_months        False
lender_count          False
tags                   True
borrower_genders       True
repayment_interval    False
date                  False
dtype: bool

- The use column contains Null values and since it contains long strings and the activity and sector columns pretty much summarises them, I'll drop this column instead of trying to fill it out.
- The country_code column also contains Null values but country column does not, which means that we can find out which countries are missing value a corresponding country code and fill them out.
- The region column also contains Null values. Since this column might be useful in finding out which regions per country get loans, we'll try to find a way of filling them. Like using the most common region from that particular country, otherwise drop if not possible.
- The partner_id column also contains Null values. These column might come in handy when working out how partners might help Kiva better understand their customers so I'll keep it. Dropping all data points with Nulls is quite considerable and would lead to a very high data loss, so I'll put that as a last resort.
- The funded_time column also has Nulls. It represents the time a customer received money from a partner.
- Since the tags column only contains string that somewhat are really hard to fill in, I'll drop this column.

In [155]:
loans = loans.drop('use', axis=1)

In [156]:
loans.loc[loans['country_code'].isnull(), 'country'].unique()

array(['Namibia'], dtype=object)

In [157]:
# Only Namibia is missing a correspodning country code
# From a Google search, I got that the country's ISO country code is NAM
# So I'll fill that in.
loans.loc[loans['country'] == 'Namibia', 'country_code'] = 'NAM'

In [158]:
# Number of Nulls in the region column
loans.loc[loans['region'].isnull(), 'country'].count()

56800

There's a total of 671205 data points in the entire dataset so just dropping these values (56800 of them) is not an option.

In [159]:
# Finding most common region in a country 

In [160]:
country_regions = loans.groupby(['country', 'region']).count()

In [161]:
country_regions.head()

id  funded_amount  loan_amount  activity  sector  \
country     region                                                             
Afghanistan Kandahar City    2              2            2         2       2   
Albania     Cerrik           1              1            1         1       1   
            Elbasan        243            243          243       243     243   
            Korce          753            753          753       753     753   
            Lac            122            122          122       122     122   

                           country_code  currency  partner_id  posted_time  \
country     region                                                           
Afghanistan Kandahar City             2         2           2            2   
Albania     Cerrik                    1         1           1            1   
            Elbasan                 243       243         243          243   
            Korce                   753       753         753          753   
            Lac                     122       122         122          122   

                           disbursed_time  funded_time  term_in_months  \
country     region                                                       
Afghanistan Kandahar City               2            2               2   
Albania     Cerrik                      1            1               1   
            Elbasan                   243          206             243   
            Korce                     753          661             753   
            Lac                       122          109             122   

                           lender_count  tags  borrower_genders  \
country     region                                                
Afghanistan Kandahar City             2     2                 2   
Albania     Cerrik                    1     1                 1   
            Elbasan                 243   224               243   
            Korce                   753   698               753   
            Lac                     122   120               122   

                           repayment_interval  date  
country     region                                   
Afghanistan Kandahar City                   2     2  
Albania     Cerrik                          1     1  
            Elbasan                       243   243  
            Korce                         753   753  
            Lac                           122   122

In [162]:
ids = country_regions['id']

In [163]:
ids.head()

country      region       
Afghanistan  Kandahar City      2
Albania      Cerrik             1
             Elbasan          243
             Korce            753
             Lac              122
Name: id, dtype: int64

In [164]:
ids_counts = ids.max(level='country')

In [165]:
ids_counts.head()

country
Afghanistan       2
Albania         753
Armenia        1022
Azerbaijan      845
Belize           44
Name: id, dtype: int64

In [166]:
countries = loans['country'].unique()

In [167]:
most_common_countries_regions = {country: None for country in countries}

In [18]:
for i in range(ids.size):
    for j in range(ids_counts.size):
        if ids.iloc[i] == ids_counts.iloc[j] and ids.index[i][0] == ids_counts.index[j]:
            most_common_countries_regions[ids_counts.index[j]] = ids.index[i][1]
            break

In [19]:
most_common_countries_regions

{'Pakistan': 'Lahore',
 'India': 'Dhupguri',
 'Kenya': 'Kisii',
 'Nicaragua': 'Managua',
 'El Salvador': 'San Miguel',
 'Tanzania': 'Dar es Salaam',
 'Philippines': 'Palo, Leyte',
 'Peru': 'Cusco',
 'Senegal': 'Ronkh',
 'Cambodia': 'Battambang',
 'Liberia': 'New Kru',
 'Vietnam': 'Thanh Hoá',
 'Iraq': None,
 'Honduras': 'San Lorenzo',
 'Palestine': 'Rafah city, Gaza Strip',
 'Mongolia': 'Ulaanbaatar',
 'United States': 'San Diego',
 'Mali': 'Bamako',
 'Colombia': 'Medellín',
 'Tajikistan': 'Dushanbe',
 'Guatemala': 'Solola',
 'Ecuador': 'San Gabriel',
 'Bolivia': 'La Paz',
 'Yemen': "Sana'a",
 'Ghana': 'Teshie, Accra',
 'Sierra Leone': 'Freetown East',
 'Haiti': 'Fontamara(Centre-Ville)',
 'Chile': 'Cañete',
 'Jordan': 'Irbid',
 'Uganda': 'Kampala',
 'Burundi': 'Bujumbura',
 'Burkina Faso': 'Gounghin, Ouagadougou',
 'Timor-Leste': 'Oe-cusse',
 'Indonesia': 'Tangerang',
 'Georgia': 'Tbilisi',
 'Ukraine': 'Vinogradovo',
 'Kosovo': None,
 'Albania': 'Korce',
 'The Democratic Republic of t

- Was able to get the most common regions per country but couldn't come up with a way to fill in all the Null values in the region columns with repect to most common region for it's respective country. I'll just drop them an continue with my analysis.
- I don't really recommend this considering how much data will be lost but it's my only option at the moment.

In [168]:
loans = loans.dropna(subset=['region'])

In [169]:
# partner_id
loans.loc[loans['partner_id'].isnull()].head()

,id,funded_amount,loan_amount,activity,sector,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date


In [170]:
# Number of partner_id missing
loans.loc[loans['partner_id'].isnull(), 'country'].count()

0

- Trying to fill in these Nulls would require the same steps as trying to fill in the Nulls of regions, so until I find a solution to that problem I'll have to drop them.

In [171]:
loans = loans.dropna(subset=['partner_id'])

In [172]:
# funded_time
loans.loc[loans['funded_time'].isnull(), 'country'].count()

40278

- Judging from the number and the amount of data points left. I'll leave it as it is.

In [173]:
# tags
loans = loans.drop('tags', axis=1)

In [174]:
# check for nulls
loans.isnull().any()

id                    False
funded_amount         False
loan_amount           False
activity              False
sector                False
country_code          False
country               False
region                False
currency              False
partner_id            False
posted_time           False
disbursed_time        False
funded_time            True
term_in_months        False
lender_count          False
borrower_genders      False
repayment_interval    False
date                  False
dtype: bool

- Since most of the Nulls have been dealt with. I'll move on to customer analysis.

# Customer Analysis 

In [175]:
loans.head()

,id,funded_amount,loan_amount,activity,sector,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,female,monthly,2014-01-01


In [176]:
# sectors potentially describing the reason why money is borrowed.
loans['sector'].unique()

array(['Food', 'Transportation', 'Arts', 'Agriculture', 'Services',
       'Manufacturing', 'Wholesale', 'Retail', 'Clothing', 'Construction',
       'Health', 'Education', 'Personal Use', 'Housing', 'Entertainment'],
      dtype=object)

In [177]:
# The number of sectors.
len(loans['sector'].unique())

15

In [178]:
# Total funded amount per sector
funded_amount_per_sector = loans.groupby(['sector'])['funded_amount'].sum()
funded_amount_per_sector.sort_values(ascending=False)

sector
Agriculture       122953400.0
Food               95321825.0
Retail             78270625.0
Services           34272400.0
Clothing           30198600.0
Education          27828200.0
Housing            18496725.0
Personal Use       13748900.0
Arts               10142600.0
Transportation      9263525.0
Health              8236700.0
Construction        5498425.0
Manufacturing       4858000.0
Wholesale            865675.0
Entertainment        608150.0
Name: funded_amount, dtype: float64

- Most funding goes towards supporting Agriculture related activities. So I can assume that most customers are food farmers on average.
- The least funding goes to Entertainment sector.

In [179]:
# Most funded sector per country. This should help in understaing which economic sector
# most customers are engaged in per country. This can more accurately be found by counting
# the number of ids, but I'll be using funded_amount column.
country_sector = loans.groupby(['country', 'sector']).sum()
country_sector

id  funded_amount  loan_amount  partner_id  \
country     sector                                                              
Afghanistan Arts              1829155        14000.0      14000.0       824.0   
Albania     Agriculture     561641603       837775.0     865525.0    147224.0   
            Arts              3806186         6725.0       6725.0       956.0   
            Clothing         62563243       120900.0     128450.0     15372.0   
            Construction     23482091        32675.0      35000.0      6214.0   
...                               ...            ...          ...         ...   
Zimbabwe    Personal Use       959926         1400.0       1400.0       367.0   
            Retail          728415184       757525.0     790150.0    241858.0   
            Services        228551103       209025.0     219775.0     73694.0   
            Transportation    8988325        11550.0      11550.0      3320.0   
            Wholesale        14531442        22300.0      22300.0      5748.0   

                            term_in_months  lender_count  
country     sector                                        
Afghanistan Arts                      14.0           512  
Albania     Agriculture            14737.0         27847  
            Arts                      90.0           231  
            Clothing                1599.0          3969  
            Construction             562.0          1073  
...                                    ...           ...  
Zimbabwe    Personal Use               8.0            26  
            Retail                  7074.0         22010  
            Services                2133.0          6435  
            Transportation            79.0           394  
            Wholesale                137.0           692  

[915 rows x 6 columns]

In [180]:
funded_amount_counts = country_sector['funded_amount']
funded_amount_counts

country      sector        
Afghanistan  Arts               14000.0
Albania      Agriculture       837775.0
             Arts                6725.0
             Clothing          120900.0
             Construction       32675.0
                                 ...   
Zimbabwe     Personal Use        1400.0
             Retail            757525.0
             Services          209025.0
             Transportation     11550.0
             Wholesale          22300.0
Name: funded_amount, Length: 915, dtype: float64

In [181]:
funded_amount_counts_top = funded_amount_counts.max(level='country')
funded_amount_counts_top

country
Afghanistan      14000.0
Albania         837775.0
Armenia        6603425.0
Azerbaijan     1472500.0
Belize          111975.0
                 ...    
Vanuatu           9250.0
Vietnam        4714375.0
Yemen           398550.0
Zambia          697775.0
Zimbabwe        808500.0
Name: funded_amount, Length: 82, dtype: float64

In [182]:
top_funded_sector_country = {country: None for country in countries}

In [183]:
for i in range(funded_amount_counts.size):
    for j in range(funded_amount_counts_top.size):
        if funded_amount_counts.iloc[i] == funded_amount_counts_top.iloc[j] and funded_amount_counts.index[i][0] == funded_amount_counts_top.index[j]:
            top_funded_sector_country[funded_amount_counts_top.index[j]] = funded_amount_counts.index[i][1]
            break

In [184]:
top_funded_sector_country

{'Pakistan': 'Services',
 'India': 'Agriculture',
 'Kenya': 'Agriculture',
 'Nicaragua': 'Food',
 'El Salvador': 'Agriculture',
 'Tanzania': 'Food',
 'Philippines': 'Retail',
 'Peru': 'Food',
 'Senegal': 'Agriculture',
 'Cambodia': 'Agriculture',
 'Liberia': 'Food',
 'Vietnam': 'Agriculture',
 'Iraq': None,
 'Honduras': 'Agriculture',
 'Palestine': 'Education',
 'Mongolia': 'Housing',
 'United States': 'Services',
 'Mali': 'Agriculture',
 'Colombia': 'Agriculture',
 'Tajikistan': 'Agriculture',
 'Guatemala': 'Agriculture',
 'Ecuador': 'Agriculture',
 'Bolivia': 'Food',
 'Yemen': 'Personal Use',
 'Ghana': 'Food',
 'Sierra Leone': 'Food',
 'Haiti': 'Retail',
 'Chile': 'Health',
 'Jordan': 'Education',
 'Uganda': 'Agriculture',
 'Burundi': 'Food',
 'Burkina Faso': 'Food',
 'Timor-Leste': 'Retail',
 'Indonesia': 'Agriculture',
 'Georgia': 'Agriculture',
 'Ukraine': 'Agriculture',
 'Kosovo': None,
 'Albania': 'Agriculture',
 'The Democratic Republic of the Congo': 'Food',
 'Costa Rica': 'Ag

- The above dictionary shows the most funded sector per country, although some contain None which might be due to dropping of Nulls in the initial section.

In [185]:
# Countries with the highest and lowest total lender count
lender_counts = loans.groupby(['country'])['lender_count'].sum()
lender_counts

country
Afghanistan       512
Albania         68750
Armenia        320395
Azerbaijan      76336
Belize           4106
                ...  
Vanuatu           340
Vietnam        404103
Yemen           48074
Zambia          35983
Zimbabwe        99241
Name: lender_count, Length: 82, dtype: int64

In [186]:
max_count = lender_counts.max()
min_count = lender_counts.min()

In [187]:
for i in range(lender_counts.size):
    if lender_counts.iloc[i] == max_count:
        max_country = lender_counts.index[i]
    elif lender_counts.iloc[i] == min_count:
        min_country = lender_counts.index[i]

In [188]:
print(f"Highest Total Lender Counts: {max_country}\nLowest Total Lender Counts: {min_country}")

Highest Total Lender Counts: Philippines
Lowest Total Lender Counts: Vanuatu


In [189]:
# Borrowers genders
loans['borrower_genders'].unique()

array(['female', 'female, female', 'female, female, female', ...,
       'female, female, female, female, male, male, female, female, female, female, female, female',
       'male, female, female, female, female, female, female, female, male, male, female, male, female, male, male, male',
       'female, female, female, male, female, female, female, male, female, female, female, male, female, male, female, female, female, female, female, female, female, female, female, female, female, female, female, female, male'],
      dtype=object)

- Based on the dataset description, each party per data point represents a single individual, such that a data point having 'male, female' represents two people. This makes my analysis a little bit challenging, especially when trying to represent which genders on average do what. So what I'll do is sort of truncate each data point only to the first party so as to only have two unique genders.

In [190]:
loans = loans.dropna(subset=['borrower_genders'])

In [191]:
borrower_genders = [_[0] for _ in loans['borrower_genders'].str.replace(",", " ").str.split()]

In [192]:
borrower_genders = pd.Series(borrower_genders)

In [193]:
loans['borrower_genders'] = borrower_genders

In [195]:
loans = loans.dropna(subset=['borrower_genders'])

In [196]:
loans['borrower_genders'].unique()

array(['female', 'male'], dtype=object)

In [198]:
# ^^^ should got to data cleaning section

In [201]:
# Amount per gender
loans.groupby(['borrower_genders']).sum()

,id,funded_amount,loan_amount,partner_id,term_in_months,lender_count
borrower_genders,,,,,,
female,420712393999,331681475.0,347434750.0,77794600.0,5988160.0,8826779
male,120766683799,96868850.0,101427575.0,22473617.0,1748122.0,2593772


In [202]:
# Total number for each gender
loans.groupby(['borrower_genders']).count()

,id,funded_amount,loan_amount,activity,sector,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,repayment_interval,date
borrower_genders,,,,,,,,,,,,,,,,,
female,434360,434360,434360,434360,434360,434360,434360,434360,434360,434360,434360,434360,408885,434360,434360,434360,434360
male,126205,126205,126205,126205,126205,126205,126205,126205,126205,126205,126205,126205,118709,126205,126205,126205,126205


In [210]:
# Sector which majority for each gender are invlolved
loans.groupby(['borrower_genders', 'sector'])['id'].count()

borrower_genders  sector        
female            Agriculture       118760
                  Arts                7301
                  Clothing           20375
                  Construction        4055
                  Education          20899
                  Entertainment        451
                  Food               88161
                  Health              5646
                  Housing            19514
                  Manufacturing       4142
                  Personal Use       24699
                  Retail             81240
                  Services           27947
                  Transportation     10742
                  Wholesale            428
male              Agriculture        34717
                  Arts                2201
                  Clothing            5980
                  Construction        1147
                  Education           6138
                  Entertainment        128
                  Food               25489
                  Hea

- There are more female cutomers compared to male, and they also ask for more funding.
- It is also evident that all sectors contain more female than male since the data contains more female customers than male.

In [206]:
# repayment intervals
loans['repayment_interval'].unique()

array(['irregular', 'bullet', 'monthly'], dtype=object)

In [208]:
loans.groupby(['repayment_interval'])['id'].count()

repayment_interval
bullet        59700
irregular    221534
monthly      279331
Name: id, dtype: int64

- There are three forms of repayment intervals, which are as above.
- The most common form is the monthly form, followed by the irregular by a small margin, then bullet.